## Prerequisites

In [1]:
!pip install pandas
!pip install sentencepiece
!pip install hgtk
!pip install gluonnlp

     |████████████████████████████████| 1.1MB 1.1MB/s 
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=90829c1abb57e4fece62d374291acbe8ac91b59ffeca4a3e6a61dcfc0a4d9072
  Stored in directory: /root/.cache/pip/wheels/73/72/06/6065a57fe68264f35d7e52e37f56831eb3e9ec75656880de20
Successfully built hgtk
     |████████████████████████████████| 348kB 2.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588518 sha256=442ed09c75a6f7d6925401908027703ee2347a2f6be2a9e3a18a70cd07a930ba
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp


## Base Function

In [1]:
def file_num_padding(file_num) :
    if file_num < 10 :
        return '00000' + str(file_num)
    elif file_num < 100 :
        return '0000' + str(file_num)
    elif file_num < 1000 :
        return '000' + str(file_num)
    elif file_num < 10000 :
        return '00' + str(file_num)
    elif file_num < 100000 :
        return '0' + str(file_num)
    else :
        return str(file_num)

In [2]:
def get_path(path, fname, file_num, format):
    return path + fname +file_num + format

In [5]:
BASE_PATH = './content/drive/My Drive/googledrive/'
fname = 'KsponSpeech_'
file_num = 1
format = '.txt'

print(get_path(BASE_PATH, fname, file_num_padding(file_num), format))

./content/drive/My Drive/googledrive/KsponSpeech_000001.txt


## preprocess

In [6]:
import os
import re

In [7]:
# mode = spelling(e.g., 70%는 칠십퍼센트 이렇게 표기되는 게 아니라 70%로 표기되도록)
def bracket_filter(sentence):
    # new_sentence = str()
    new_sentence = ""
    flag = True

    for ch in sentence :
        if ch == '(' :
            continue

        if ch == ')' :
            if flag is True :
                flag = False
                continue
            else :
                flag = True
                continue

        if ch != ')' and flag is True :
            new_sentence += ch

            elif not skip :
                new_sentence += ch

        return new_sentence

SyntaxError: ignored

## Special Filter

In [8]:
def special_filter(sentence, mode='phonetic', replace=None):
    SENTENCE_MARK = ['?', '!', '.']

    # noise라고 인식하면 안 되는 것들 추려내기!
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', ',']

    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx + 1] == '/':
                continue

        if ch == '#':
            new_sentence += '샾'

        elif ch == '%':
            if mode == 'phonetic':
                new_sentence += replace
            elif mode == 'spelling':
                new_sentence += '%'

        elif ch not in EXCEPT:
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence


def sentence_filter(raw_sentence, mode, replace=None):
    return special_filter(bracket_filter(raw_sentence, mode), mode, replace)


def preprocess(dataset_path, new_path, mode='phonetic'):
    print('preprocess started..')

    percent_files = {
        '087797': '퍼센트',
        '215401': '퍼센트',
        '284574': '퍼센트',
        '397184': '퍼센트',
        '501006': '프로',
        '502173': '프로',
        '542363': '프로',
        '581483': '퍼센트'
    }

    for folder in os.listdir(dataset_path):
        # folder : {KsponSpeech_01, ..., KsponSpeech_05}
        if not folder.startswith('KsponSpeech'):
            continue
        path = os.path.join(dataset_path, folder)
        for idx, subfolder in enumerate(os.listdir(path)):
            if idx == 0:
                if not (os.path.isdir(os.path.join(new_path, folder))):
                    os.makedirs(os.path.join(new_path, folder))
            path = os.path.join(dataset_path, folder, subfolder)

            for jdx, file in enumerate(os.listdir(path)):
                if jdx == 0:
                    if not (os.path.isdir(os.path.join(new_path, folder, subfolder))):
                        os.makedirs(os.path.join(new_path, folder, subfolder))

                if file.endswith('.txt'):
                    with open(os.path.join(path, file), "r", encoding='cp949') as f:
                        raw_sentence = f.read()
                        if file[12:18] in percent_files.keys():
                            new_sentence = sentence_filter(raw_sentence, mode, percent_files[file[12:18]])
                        else:
                            new_sentence = sentence_filter(raw_sentence, mode=mode)

                    with open(os.path.join(new_path, folder, subfolder, file), "w", encoding='cp949') as f:
                        f.write(new_sentence)

                else:
                    continue
